In [1]:
import pandas as pd
import os
import dataflow
import datetime
import trade_strategy
from dateutil.relativedelta import relativedelta
from trade_logger import TradeLogger, EventType
%matplotlib inline

In [2]:
df = pd.read_excel("./crawl_result/ETF50/1101.xlsx")

In [3]:
df = dataflow.append_all_index(dataflow.preprocess(df))

--
--
--
--
--
--


/Users/rogerlo/Documents/work/code/python/test/finace_crawler/toolbox.py:78: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  rsv_series = df[column].rolling(window).apply(RSV)


In [4]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    money_in_pocket = 100 #(K)
    money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.baseline(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i))
    print(trade_log)
    #報酬
    diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
    #報酬率
    print("year: {}, diff: {:f}, diff%: {:f}".format((start_date+relativedelta(years=i)).year, diff, diff/money_in_pocket*100))
    

2013-01-01 00:00:00: EventType.BUY_STOCK shares:2 price:39.000000 money_in_pocket:22.000000
2013-07-25 00:00:00: EventType.GET_INTEREST shares:2 price:1.900000 money_in_pocket:25.800000
year: 2013, diff: 18.300000, diff%: 18.300000
2014-01-01 00:00:00: EventType.BUY_STOCK shares:2 price:45.400000 money_in_pocket:9.200000
2014-07-17 00:00:00: EventType.GET_INTEREST shares:2 price:2.300000 money_in_pocket:13.800000
year: 2014, diff: 0.600000, diff%: 0.600000
2015-01-01 00:00:00: EventType.BUY_STOCK shares:2 price:43.000000 money_in_pocket:14.000000
2015-07-23 00:00:00: EventType.GET_INTEREST shares:2 price:2.490000 money_in_pocket:18.980000
year: 2015, diff: -26.420000, diff%: -26.420000
2016-01-01 00:00:00: EventType.BUY_STOCK shares:3 price:26.150000 money_in_pocket:21.550000
2016-07-25 00:00:00: EventType.GET_INTEREST shares:3 price:1.330000 money_in_pocket:25.540000
year: 2016, diff: 30.990000, diff%: 30.990000
2017-01-01 00:00:00: EventType.BUY_STOCK shares:2 price:35.150000 money_i

In [5]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    money_in_pocket = 100 #(K)
    money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.KD_basic(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i))
    print(trade_log)
    #報酬
    diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
    #報酬率
    print("year: {}, diff: {:f}, diff%: {:f}".format((start_date+relativedelta(years=i)).year, diff, diff/money_in_pocket*100))
    

2013-03-21 00:00:00: EventType.BUY_STOCK shares:1 price:36.500000 money_in_pocket:63.500000
2013-03-22 00:00:00: EventType.BUY_STOCK shares:1 price:36.450000 money_in_pocket:27.050000
2013-04-17 00:00:00: EventType.SELL_STOCK shares:1 price:38.400000 money_in_pocket:65.450000
2013-04-19 00:00:00: EventType.SELL_STOCK shares:1 price:38.400000 money_in_pocket:103.850000
2013-06-07 00:00:00: EventType.BUY_STOCK shares:1 price:37.850000 money_in_pocket:66.000000
2013-06-13 00:00:00: EventType.BUY_STOCK shares:1 price:37.350000 money_in_pocket:28.650000
2013-07-15 00:00:00: EventType.SELL_STOCK shares:1 price:38.000000 money_in_pocket:66.650000
2013-07-16 00:00:00: EventType.SELL_STOCK shares:1 price:38.100000 money_in_pocket:104.750000
2013-07-25 00:00:00: EventType.GET_INTEREST shares:0 price:1.900000 money_in_pocket:104.750000
2013-11-06 00:00:00: EventType.BUY_STOCK shares:1 price:40.800000 money_in_pocket:63.950000
2013-11-07 00:00:00: EventType.BUY_STOCK shares:1 price:41.050000 money